# Take 2

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import statsmodels.api as sm
import pmdarima as pm
import warnings
from sklearn.metrics import mean_squared_error
import math
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [3]:
def make_stock_data(stocks):
    info_list=[]
    # Close
    for stock in stocks:
        info = yf.Ticker(stock).history(period='max')[['Close']]
        info_list.append(info)
    combo = pd.concat(info_list,axis=1)
    combo.columns = stocks

    combo = combo.dropna()
    combo.index = pd.to_datetime(combo.index)
    return combo

In [23]:
stock_name = ['pcar','amzn','tsla','oil','aapl']

combo = make_stock_data(stock_name)
combo.head(10)


,pcar,amzn,tsla,oil,aapl
Date,,,,,
2011-04-21,35.873421,9.2945,1.782667,50.349998,10.694245
2011-04-25,35.388557,9.2710,1.759333,50.049999,10.764687
2011-04-26,36.338085,9.1150,1.795333,50.299999,10.685703
2011-04-27,36.573795,9.8315,1.805333,50.810001,10.677473
2011-04-28,36.196671,9.7535,1.844000,50.700001,10.573791
2011-04-29,35.765682,9.7905,1.840000,51.040001,10.676863
2011-05-02,35.745476,10.0595,1.830000,50.950001,10.559461
2011-05-03,35.698330,9.9225,1.791333,49.820000,10.618011
2011-05-04,35.880150,9.9985,1.779333,48.889999,10.659787


In [30]:
outs={}
for stock in stock_name:
    combo['predict'] = combo[stock].shift(-1)

    model = sm.tsa.statespace.SARIMAX(endog = combo.predict,exog = combo.drop('predict',axis=1),order=[5,0,0],seasonal_order=[0,0,0,5],trend='ct',)
    model_fit = model.fit(disp=False)
    output = model_fit.predict(exog = combo.drop('predict',axis=1))
    combo['predicted'] = output

    out = combo[[stock,'predict','predicted']].rename(columns = {stock:'Today','predict':'Tomorow Actual','predicted':'Tomorow Pred'})

    outs[stock] = out


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so 

In [31]:
outs['aapl']

,Today,Tomorow Actual,Tomorow Pred
Date,,,
2011-04-21,10.694245,10.764687,10.717055
2011-04-25,10.764687,10.685703,10.790159
2011-04-26,10.685703,10.677473,10.709860
2011-04-27,10.677473,10.573791,10.688966
2011-04-28,10.573791,10.676863,10.600788
...,...,...,...
2022-09-27,151.759995,149.839996,151.414646
2022-09-28,149.839996,142.479996,149.623877
2022-09-29,142.479996,138.199997,142.710240
